<a href="https://colab.research.google.com/github/guybanbo/Scloud/blob/main/Scloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.chat.util import Chat, reflections
!pip install requests
!pip install beautifulsoup4
!pip install firebase
!pip install ipython
!pip install langdetect
!pip install nltk
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=2af243605476757f9f8607be518c76b92e4b18ee20df9feef6dadd2f6e67563a
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

crawler

In [ ]:

import requests
from firebase import firebase
from bs4 import BeautifulSoup
from collections import defaultdict
import json
import time
from urllib.parse import urljoin, urlparse


import requests
from urllib.parse import urlparse, urljoin
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')



def is_word(text):
    return isEnglish(text) and text.isalpha() and len(text) > 2

def isEnglish(s):
  return s.isascii()

def index_words(soup):
    index = {}
    words = []

    def extract_words_from_tag(tag):
        if tag.name in ["script", "style"]:  # Exclude script and style tags
            return
        if tag.name in ["p", "h1", "h2", "h3", "h4", "h5", "h6", "span","strong"]:  # Include only these tags
            text = tag.get_text(separator=" ").strip(".,")
            words.extend(filter(is_word, text.split()))
        for child in tag.children:
            if hasattr(child, 'children'):  # Check if the child has children (nested tags)
                extract_words_from_tag(child)

    extract_words_from_tag(soup)

    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

def remove_stop_words(index):
    stop_words = set(stopwords.words('english'))
    stop_words.discard("what")  # Remove "what" from stop words if necessary
    for stop_word in stop_words:
        index.pop(stop_word, None)
    return index


def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, docs in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
          stemmed_index[stemmed_word] = list(set(stemmed_index[stemmed_word] + docs))
        else:
          stemmed_index[stemmed_word] = docs
    return stemmed_index


#extracting pages' titles and descriptions
def links_enrichment (links):
  meta=[]
  for link in links:
    try:
        # Fetch the page content
        time.sleep(1)
        # Create a session object to manage redirections
        session = requests.Session()
        session.max_redirects = 10
        #response = requests.get(link,timeout=10)
        response = session.get(link, timeout=10)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract meta description
        meta_description = soup.find('meta', {'name': 'description'})
        description = meta_description['content'] if meta_description else "no description available"

        # Extract title
        title_tag = soup.find('title')
        title = title_tag.text if title_tag else link

        meta.append( {"link" : link,
                "title" : title,
                "description" : description} )
    except requests.Timeout:
        print("Timeout crawling {}".format(link))
    except Exception as e:
        meta.append( {"link" : link,
                "title" : link,
                "description" : "no description available"})
        print(e)
  return meta


def create_index(links):

    word_index = {}
    for id,data in links.items():
      # Fetch the page content
      try:
        # Create a session object to manage redirections for this URL
        time.sleep(1)
        session = requests.Session()
        session.max_redirects = 10

            # Fetch the page content using the session object
        response = session.get(data["link"],timeout=30)
        #response = requests.get(data["link"])
        response.raise_for_status()

       # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        words = index_words(soup)
        words = remove_stop_words(words)
        for word in words:
         # Append the link(id) to the list of links for each word
         word_index.setdefault(word, []).append(id)
      except requests.Timeout:
        print("Timeout crawling {}".format(data["link"]))
      except Exception as e:
          print("Error INDEXING {}: {}".format(data, e))
    return word_index


# Function to delete a db table
def delete_node(node_path):
    delete_url = f"{url}/{node_path}.json"
    response = requests.delete(delete_url)
    if response.status_code == 200:
        print(f"Node '{node_path}' deleted successfully")
    else:
        print(f"Failed to delete node '{node_path}', Status code: {response.status_code}")



problematic_urls=["https://topic.alibabacloud.com/zqpop/","https://www.alibabacloud.com/forum/","https://www.alibabacloud.com/ja",
                  "https://www.alibabacloud.com/tc","https://www.alibabacloud.com/zh", "https://www.alibabacloud.com/help/zh/","https://account.alibabacloud.com/"]
visited = set()
# Function to crawl a URL up to a certain depth
def crawl(url, max_depth, depth=0):
    if depth > max_depth:
        return

    if url in visited:
        return

    visited.add(url)
    # Create a session object to manage redirections
    session = requests.Session()
    session.max_redirects = 10
    try:
        # Fetch the page content
        time.sleep(1)
        response = session.get(url, timeout=10)
        #response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract links
        for link in soup.find_all('a', href=True):
            href = link['href']
            parsed_href = urlparse(href)
            domain = parsed_href.netloc
            valid=True
            for i in problematic_urls:
              if href.startswith(i):
                valid=False
                break
            if parsed_href.scheme and domain and (domain.endswith('.alibabacloud.com') or domain == 'alibabacloud.com') and valid==True: # Check if it's a valid URL
                absolute_url = urljoin(url, href)
                if absolute_url not in visited:#added check to reduce recursive calls
                  crawl(absolute_url, max_depth, depth + 1)
    except requests.Timeout:
        print("Timeout crawling {}".format(url))
    except Exception as e:
        print("Error crawling {}: {}".format(url, e))


"""
# Start crawling from the root URL
root_url = "https://www.alibabacloud.com/"
max_depth = 1
crawl(root_url, max_depth)
crawled_links =list(visited)
links = links_enrichment(crawled_links)
"""

url = "https://search-engine-568ee-default-rtdb.europe-west1.firebasedatabase.app"
FBconn = firebase.FirebaseApplication(url,None)
"""
delete_node('pages2')
delete_node('index2')
for link in links:
  FBconn.post('/pages2/',link)
result = FBconn.get('/pages2', None)
index = create_index(result)
stemmed_Index=apply_stemming(index)
for term,DocIDs in stemmed_Index.items():
  data = {"term":term, "DocIDs": DocIDs}
  FBconn.post('/index2/',data)
"""







[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'\ndelete_node(\'pages2\')\ndelete_node(\'index2\')\nfor link in links:\n  FBconn.post(\'/pages2/\',link)\nresult = FBconn.get(\'/pages2\', None)\nindex = create_index(result)\nstemmed_Index=apply_stemming(index)\nfor term,DocIDs in stemmed_Index.items():\n  data = {"term":term, "DocIDs": DocIDs}\n  FBconn.post(\'/index2/\',data)\n'

**search engine**

In [ ]:
# Import necessary modules
from IPython.display import display, HTML
import nltk
from nltk.chat.util import Chat, reflections
from google.colab import output



# Define some patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me Alibot.', 'I go by the name Alibot.']),
    (r'(.*)(explain|what) (on|about|is) Alibaba cloud?', ['Alibaba Cloud is a leading cloud service provider offering a range of services including computing, storage, and databases.$https://www.alibabacloud.com']),
    (r'(.*)(explain|what) (on|about|is) public cloud?', ['Public cloud is a cloud infrastructure provisioned for use by the general public, individuals or businesses. Essentially, third-party public cloud providers lease servers within a virtual environment, which is accessible over Internet.$https://www.alibabacloud.com/knowledge/what-is-public-cloud']),
    (r'(.*)(explain|what) (on|about|is) private cloud?', ['Different from public cloud, a private cloud is a model of cloud computing that provides a dedicated cloud environment to a single organization. Private cloud can be operated internally by an in-house IT team, behind that organization’s firewall, so an organization has more control of its compute resources.$https://www.alibabacloud.com/knowledge/what-is-private-cloud']),
    (r'(.*)(explain|what) (on|about|is) hybrid cloud?', ['Hybrid cloud is a cloud computing model that combines one or more public cloud and private cloud environments through a network connection, which allows for sharing of data and applications between the different cloud environments.$https://www.alibabacloud.com/knowledge/what-is-hybrid-cloud']),
    (r'your name (.*)', ['You can call me Alibot.', 'I go by the name Alibot.']),
    (r'(.*)(explain|what) (about|is) saas', ["Software as a service (SaaS) allows users to connect to and use cloud-based apps over the Internet.$https://www.alibabacloud.com/blog/597477?spm=a3c0i.23458820.2359477120.9.79047d3fX50A6a"]),
    (r'What kinds of clouds are there ?', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'(.*)kinds of clouds(.*)?(.*)', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'(explain|what) (on|about|is) paas', ['Platform as a service (PaaS) is a form of cloud computing that provides a platform allowing consumers to develop and deploy applications and services over Wide Area Network (WAN) or the Internet.$https://www.alibabacloud.com/knowledge/what-is-paas?spm=a3c0i.23458820.2359477120.120.79047d3fX50A6a']),
    (r'(explain|what) (on|about|is) iaas', ["Infrastructure as a service (IaaS) is a form of cloud computing based on automation and orchestration technologies to on-demand provision and manage virtualized infrastructure resources such as server, storage and network over a wide area network (WAN) to the consumer.$https://www.alibabacloud.com/knowledge/what-is-iaas"]),
    (r'(.*)(kinds of|types of) clouds', ['There are various types of clouds, such as public, private, and hybrid clouds. Would you like to know more about a specific type?']),

]

# Create a chatbot
chatbot = Chat(patterns, reflections)



output.register_callback("notebook.askChat",chatbot.respond)




class HTML:
    def __init__(self, code):
        self.code = code

    def render(self):
        return self.code


class CSS:
    def __init__(self, style):
        self.style = style

    def render(self):
        return f'<style>{self.style}</style>'


class JS:
    def __init__(self, code):
        self.code = code

    def render(self):
        return f'<script>{self.code}</script>'


class Main:
    def __init__(self):
        self.css = CSS("""
   body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
      background: linear-gradient(4deg, rgb(228 255 254 / 71%) 100%, rgb(211 255 252 / 70%) 128.13%, rgb(232 255 254 / 42%) 142.19%, rgb(224 255 253 / 40%));
      zoom: 80%;
  }
  header {
    background-color: rgb(154 232 225);
    color: white;
    padding: 10px 20px;
    display: flex;
    justify-content: space-between;
    align-items: center;
  }
  footer{
    background: linear-gradient(287.56deg, rgba(30, 233, 221, 0.7099999785) 100%, rgba(72, 255, 244, 0.6979166865) 128.13%, rgba(98, 255, 245, 0.4235678911) 142.19%, rgba(91, 255, 245, 0.4010416865) 154.17%, rgba(110, 255, 246, 0.4637933075) 174.48%, rgba(40, 238, 226, 0.5799999833) 200%);
  }
.logo {
  display: contents;
}

.logo img {
  max-width: 100%;
  width:50px;
  height: auto;
  margin: 0 auto; /* Center the image horizontally within the flex container */
  display: block;
}


.logo:hover {
  opacity: 0.7;
}


  .search-container {
  display: flex;
  flex-direction: column;
  align-items: center;
}

.search-input {
  padding: 10px 40px 10px 10px; /* Added padding on the right for the icon */
  width: 400px;
  font-size: 16px;
  outline: none;
  background-color: #ffffff;
  border-radius: 5rem;
  box-shadow: inset 0.4rem 0.4rem 0.4rem rgba(0, 0, 0, 0.25);
  border: 2px solid rgba(30, 233, 221, 0.71);
  /* Added background image */
  background-image: url('https://i.imgur.com/aoP2Yrs.png');
  background-repeat: no-repeat;
  background-position: right 10px center; /* Adjusted position */
  background-size: 20px; /* Adjusted size */
  margin-bottom:15px;
}

.search-button {
  display: inline-block;
  padding: 10px 20px;
  color: #00867e;
  font-size: 16px;
  border-radius: 5px;
  cursor: pointer;
  background-color: rgba(255, 255, 255, 0.5);
  border: solid 0.1rem rgba(0, 134, 126, 0.38);
  box-sizing: border-box;
  width: auto;
  height: auto;
  margin-top: 10px;
  margin-bottom:95px;
}

  .popular-searches  , .rect-searches {
    color: black;
    font-family: Roboto, 'Source Sans Pro';
    margin-top: 50px;
    text-align: center;
    border: 3px solid #b8dcda;
    border-radius: 30px;
    padding: 4px 10px 23px 10px;
    text-decoration: underline;
  }

  .rect-searches h2 {
        margin-bottom: 42px;
  }

    .popular-searches h2 {
        margin-bottom: 42px;
  }
  .popular-links {
    display: flex;
    justify-content: space-around;
    margin-top: 20px;
  }
  .popular-links a {
    text-decoration: none;
    font-weight: bold;
    padding: 10px 20px;
    border-radius: 5px;
    transition: all 0.3s ease;
    color: #1a0dab;
    font-family: Roboto, 'Source Sans Pro';
    border: 1px solid;
    background-color: white;
  }
  .popular-links a:hover {
    background-color: #008880;
    color: white;
  }
  .buttonContainer {
    display: flex;
    flex-direction: column;
    width:100px;;
}

.button {
    display: block;
    margin: 5px;
    padding: 5px 10px; /* Adjust padding to make buttons smaller */
    font-size: 12px; /* Adjust font size to make buttons smaller */
    width: auto;
}

.search-icon {
  width: auto;
  height: auto;
  margin-top: 75px;
  margin-right: 10px;
  margin-bottom: 15px; /* Space between icon and input */
}
.bookmark-button button {
  background-image: url('https://i.imgur.com/LjnGbbc.png');
  background-size: contain;
  background-color: rgba(255, 255, 255, 0.1899999976);
  width: 30px;
  height: 40px;
  border: none;
  padding: 0;
  cursor: pointer;
}


.bookmark-button button:hover {
  opacity: 0.8;
}

.indexButton {
    display: block;
    margin: 5px;
    padding: 5px 10px; /* Adjust padding to make buttons smaller */
    font-size: 12px; /* Adjust font size to make buttons smaller */
    width: auto;
      background-color: white;
      border-radius: 12px;
}

#content {
    margin-top: 20px;
}

.container {
        display: grid;
        grid-template-columns: 15% 85%; /* Set the width for each column */
        gap: 10px; /* Adjust the gap between columns */
        zoom: 125%;
}

.Scontainer {
      display: flex;
      justify-content: center;
      margin-top: 20px;
    }
    .chart {
      margin-right: 20px;
    }

.div1, .div2 {
            padding: 20px;
        background-color: #27d6cc;
        display: grid;
    place-items: center;
}

#navigationButtons {
      margin-top: 15px;
}



#links {
  /* Center the links div */
  flex: 1;
  display: flex;
  justify-content: center;
}
.admin-button,
    .stats-button {
      position: absolute;
      top: 10px;
    }

    .admin-button {
      right: 20px;
    }

    .stats-button {
      right: 80px;
    }

    .admin-button button{
      width: 40px;
      height: 40px;
      background-image: url('https://i.imgur.com/sOr8qlw.jpg');
      background-size: cover;
      background-repeat: no-repeat;
      background-position: center;
      border: none;
      cursor: pointer;
    }
    .stats-button button {
      width: 40px;
      height: 40px;
      background-image: url('https://i.imgur.com/9yrnqf7.png');
      background-size: cover;
      background-repeat: no-repeat;
      background-position: center;
      border: none;
      cursor: pointer;
    }


    .admin-button button:hover,
    .stats-button button:hover {
      opacity: 0.8;
    }

    .scrollable {
      max-height: 500px;
      overflow-y: auto;
    }

    .homeSearchesContainer{
          max-width: 60%;
          display: block;
          margin: auto;
    }


      #searchResults {
    margin-bottom: 20px;
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 8px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
  }

  .title {
    font-size: 18px;
    font-weight: bold;
    color: #1a0dab;
    margin-bottom: 5px;
  }

  .description {
    font-size: 14px;
    color: #545454;
    margin-bottom: 10px;
  }

  .link {
    font-size: 14px;
    color: #1a0dab;
    text-decoration: none;
  }

  .searchResultContainer {
    width: 80%;
    margin:auto;
  }

  .searchResultDiv {
        border-bottom: 1px solid #b6e5e2;
    margin-bottom: 25px;
  }
  #searchIndexTable {
      width: 100%;
      margin-bottom: 20px;
      border-collapse: collapse;
    }

    #searchIndexTable th, #searchIndexTable td {
      padding: 2px;
      border-bottom: 1px solid #ddd;
      text-align: center;
    }

    #searchIndexTable th {
      background-color: #f2f2f2;
    }
    * {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
  font-family: "Poppins", sans-serif;
}
    .chatbot-toggler {
  position: fixed;
  bottom: 30px;
  right: 55px;
  outline: none;
  border: none;
  height: 50px;
  width: 50px;
  display: flex;
  cursor: pointer;
  align-items: center;
  justify-content: center;
  border-radius: 50%;
  background: #724ae8;
  transition: all 0.2s ease;
}
body.show-chatbot .chatbot-toggler {
  transform: rotate(90deg);
}
.chatbot-toggler span {
  color: #fff;
  position: absolute;
}
.chatbot-toggler span:last-child,
body.show-chatbot .chatbot-toggler span:first-child  {
  opacity: 0;
}
body.show-chatbot .chatbot-toggler span:last-child {
  opacity: 1;
}
.chatbot {
  position: fixed;
  right: 35px;
  bottom: 90px;
  width: 420px;
  background: #fff;
  border-radius: 15px;
  overflow: hidden;
  opacity: 0;
  pointer-events: none;
  transform: scale(0.5);
  transform-origin: bottom right;
  box-shadow: 0 0 128px 0 rgba(0,0,0,0.1),
              0 32px 64px -48px rgba(0,0,0,0.5);
  transition: all 0.1s ease;
}
body.show-chatbot .chatbot {
  opacity: 1;
  pointer-events: auto;
  transform: scale(1);
}
.chatbot header {
  padding: 16px 0;
  position: relative;
  text-align: center;
  color: #fff;
  background: #724ae8;
  box-shadow: 0 2px 10px rgba(0,0,0,0.1);
}
.chatbot header span {
  position: absolute;
  right: 15px;
  top: 50%;
  display: none;
  cursor: pointer;
  transform: translateY(-50%);
}

.chatbot .chatbox {
  overflow-y: auto;
  height: 510px;
  padding: 30px 20px 100px;
}
.chatbot :where(.chatbox, textarea)::-webkit-scrollbar {
  width: 6px;
}
.chatbot :where(.chatbox, textarea)::-webkit-scrollbar-track {
  background: #fff;
  border-radius: 25px;
}
.chatbot :where(.chatbox, textarea)::-webkit-scrollbar-thumb {
  background: #ccc;
  border-radius: 25px;
}
.chatbox .chat {
  display: flex;
  list-style: none;
}
.chatbox .outgoing {
  margin: 20px 0;
  justify-content: flex-end;
}
.chatbox .incoming span {
  width: 32px;
  height: 32px;
  color: #fff;
  cursor: default;
  text-align: center;
  line-height: 32px;
  align-self: flex-end;
  background: #724ae8;
  border-radius: 4px;
  margin: 0 10px 7px 0;
}
.chatbox .chat p {
  white-space: pre-wrap;
  padding: 12px 16px;
  border-radius: 10px 10px 0 10px;
  max-width: 75%;
  color: #fff;
  font-size: 0.95rem;
  background: #724ae8;
}
.chatbox .incoming p {
  border-radius: 10px 10px 10px 0;
}
.chatbox .chat p.error {
  color: #721c24;
  background: #f8d7da;
}
.chatbox .incoming p {
  color: #000;
  background: #f2f2f2;
}
.chatbot .chat-input {
  display: flex;
  gap: 5px;
  position: absolute;
  bottom: 0;
  width: 100%;
  background: #fff;
  padding: 3px 20px;
  border-top: 1px solid #ddd;
}
.chat-input textarea {
  height: 55px;
  width: 100%;
  border: none;
  outline: none;
  resize: none;
  max-height: 180px;
  padding: 15px 15px 15px 0;
  font-size: 0.95rem;
}
.chat-input span {
  align-self: flex-end;
  color: #724ae8;
  cursor: pointer;
  height: 55px;
  display: flex;
  align-items: center;
  visibility: hidden;
  font-size: 1.35rem;
}
.chat-input textarea:valid ~ span {
  visibility: visible;
}

@media (max-width: 490px) {
  .chatbot-toggler {
    right: 20px;
    bottom: 20px;
  }
  .chatbot {
    right: 0;
    bottom: 0;
    height: 100%;
    border-radius: 0;
    width: 100%;
  }
  .chatbot .chatbox {
    height: 90%;
    padding: 25px 15px 100px;
  }
  .chatbot .chat-input {
    padding: 5px 15px;
  }
  .chatbot header span {
    display: block;
  }
}

}
.table-container {
  max-height: 200px; /* Adjust as needed */
  overflow-y: auto;
}

.search-index-table {
  width: 100%;
  border-collapse: collapse;
  table-layout: fixed;
}

.search-index-table th,
.search-index-table td {
  padding: 4px;
  text-align: left;
  border-bottom: 1px solid #ddd;
  white-space: nowrap;
  overflow: hidden;
  text-overflow: ellipsis;
}

.search-index-table thead th {
  position: sticky;
  top: 0;
  background-color: #f2f2f2;
  z-index: 1;
}

.search-index-table tbody th {
  font-weight: normal;
}
/***************LOGIN***********************/
  /* Adjustments for input fields */
  #loginForm label {
    display: block; /* Display labels in block format */
    margin-bottom: 10px; /* Add some space below labels */
    color: black; /* Change text color to black */
    text-align: left; /* Align labels to the left */
  }

  #loginForm input[type="text"],
  #loginForm input[type="password"] {
    width: calc(100% - 20px); /* Make input fields full width minus padding */
    padding: 10px; /* Add padding for better appearance */
    margin-bottom: 15px; /* Add some space below input fields */
    box-sizing: border-box; /* Include padding and border in element's total width and height */
  }

  #loginForm button[type="submit"] {
    width: calc(100% - 7px);
    padding: 7px; /* Adjust padding for smaller button */
    margin-top: 15px; /* Add some space above the button */
    box-sizing: border-box; /* Include padding and border in element's total width and height */
    background-color: rgba(0, 130, 120); /* Set background color to match border color */
    color: white; /* Set text color to white */
    border: 2px solid rgba(0, 130, 120); /* Set border to match background color */
  }

  /* Modal styles */
  .modal {
    display: none;
    position: fixed;
    z-index: 1;
    left: 0;
    top: 0;
    width: 100%;
    height: 100%;
    overflow: auto;
    background-color: rgba(0, 0, 0, 0.4);
  }

  .modal-content {
    background-color: #fefefe;
    margin: 15% auto;
    padding: 20px;
    border: 2px solid rgba(0, 130, 120);
    border-radius: 20px;
    width: 30%;
    text-align: center; /* Centering the content */
  }

  .close {
    color: #aaa;
    float: right;
    font-size: 28px;
    font-weight: bold;
  }

  .close:hover,
  .close:focus {
    color: black;
    text-decoration: none;
    cursor: pointer;
  }

  #loginText {
    font-size: 24px;
    font-weight: bold;
    color: #333;
    margin-bottom: 20px;
  }

  #logout {
      width: calc(100% - 20px);
      background-color: rgba(0, 80, 70);
      color: white;
      border: 2px solid rgba(0, 130, 120);
      padding: 7px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      font-size: 16px;
      margin: 15px;
      box-sizing: border-box;
      cursor: pointer;
    }
  /******************************************************/
      .searchResultDiv .category {
                font-size: 0.8em;
                background-color: #f0f0f0;
                padding: 2px 6px;
                border-radius: 4px;
                margin-left: 8px;
            }


.loader {     position: absolute;     top: 50%;     left: 50%;     transform: translate(-50%, -50%);     }

@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}
.loaderContainer {     position: relative;     min-height: 800px; /* Set your desired minimum height */  }

.spinner {
    width: 100px;
    height: 100px;
    border: 10px solid rgba(0, 0, 0, 0.1);
    border-left-color: #33a59b;
    border-radius: 50%;
    animation: spin 1s linear infinite;
  }
        """)

        self.html = HTML("""
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Search Engine</title>
<script src="https://www.gstatic.com/firebasejs/8.6.1/firebase-app.js"></script>
<script src="https://www.gstatic.com/firebasejs/8.6.1/firebase-database.js"></script>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Material+Symbols+Outlined:opsz,wght,FILL,GRAD@48,400,0,0" />
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Material+Symbols+Rounded:opsz,wght,FILL,GRAD@48,400,1,0" />
</head>
<body>
  <header>
    <div class="admin-button">
      <button id="admin"></button>
    </div>
    <div class="stats-button">
      <!-- Add onclick event to call goToStatsPage() -->
      <button id="stats" onclick="goToStatsPage()"></button>
    </div>
      <div class="logo" onclick="backToHomepage()">
        <img src="https://i.imgur.com/JkKSKGA.png" alt="Back Button" width="50" height="50">
      </div>
           <!------------------- Login Modal ------------------->
  <div id="loginModal" class="modal">
    <div class="modal-content">
      <span class="close">&times;</span>
      <h2>Admin Login</h2>
      <form id="loginForm">
        <label for="username">Username:</label>
        <input type="text" id="username" name="username" required>
        <label for="password">Password:</label>
        <input type="password" id="password" name="password" required>
        <button id="loginButton" type="submit">Login</button>
        <button id="logout" type="button" style="display: none;">Logout</button>
      </form>
    </div>
  </div>

    <!------------------------------------------------>
  </header>
  <div id="content"></div>
  <div id="bot"></div>

</body>
</html>



        """)

        self.js = JS("""
 const homepage=`<div class="search-container">
    <img src="https://i.imgur.com/HX1DIwk.png" alt="Search Icon" class="search-icon"  width="auto" height="auto">
    <input type="text" class="search-input" id="searchInput" placeholder="">
    <button id="searchButton" class="search-button">Search</button>
  </div>
  <div class="homeSearchesContainer">
  <div class="popular-searches">
    <h2>Important Terms</h2>
    <div class="popular-links">
      <a href="#">SAAS</a>
      <a href="#">IAAS</a>
      <a href="#">PAAS</a>
      <a href="#">Public Cloud</a>
      <a href="#">Private Cloud</a>
    </div>
  </div>
  <div class="rect-searches">
    <h2>Recent Searches</h2>
    <div class="popular-links">
      <a href="#">Recent Search 1</a>
      <a href="#">Recent Search 2</a>
      <a href="#">Recent Search 3</a>
      <a href="#">Recent Search 4</a>
    </div>
  </div>
  </div>`

const searchDiv=`<div class="search-container">
<img src="https://i.imgur.com/HX1DIwk.png" class="search-icon" alt="Search Icon" >
<input type="text" class="search-input" id="searchInput" placeholder="">
<button id="searchButton" class="search-button">Scloud Search</button>
</div>`

const statsScreen = `
  <h1 style="text-align:center">Search Engine Statistics</h1>
  <div class="table-container scrollable" >
    <table id="searchIndexTable" class="search-index-table">
      <thead>
        <tr>
          <th style="font-size: 18px;">Phrase</th>
          <th style="font-size: 18px;">Number of Searches</th>
          <th style="font-size: 18px;">Percentage</th>
        </tr>
      </thead>
      <tbody>
        <!-- Table rows will be added dynamically here -->
      </tbody>
    </table>
  </div>
  <div class="Scontainer">
    <div class="chart">
     <h2 style="text-align:center;">Top Searches Counter</h2>
      <canvas id="myChart" width="600" height="400"></canvas>
    </div>
    <div class="chart">
     <h2 style="text-align:center;">Top Searches Percentage</h2>
      <canvas id="myPieChart" width="500" height="400"></canvas>
    </div>
    <div class="chart">
      <h2 style="text-align:center;">The Top 10 Phrases</h2>
      <canvas id="myBarChart" width="600" height="400"></canvas>
    </div>
  </div>
`;
const bot=`<button class="chatbot-toggler">
<span class="material-symbols-rounded">mode_comment</span>
<span class="material-symbols-outlined">close</span>
</button>
<div class="chatbot">
<header>
  <h2>Chatbot</h2>
  <span class="close-btn material-symbols-outlined">close</span>
</header>
<ul class="chatbox">
  <li class="chat incoming">
    <span class="material-symbols-outlined">smart_toy</span>
    <p>Hi there 👋<br>How can I help you today?</p>
  </li>
</ul>
<div class="chat-input">
  <textarea placeholder="Enter a message..." spellcheck="false" required></textarea>
  <span id="send-btn" class="material-symbols-rounded">send</span>
</div>
</div>
`



//porter stemmer
var stemmer = (function(){
  var step2list = {
      "ational" : "ate",
      "tional" : "tion",
      "enci" : "ence",
      "anci" : "ance",
      "izer" : "ize",
      "bli" : "ble",
      "alli" : "al",
      "entli" : "ent",
      "eli" : "e",
      "ousli" : "ous",
      "ization" : "ize",
      "ation" : "ate",
      "ator" : "ate",
      "alism" : "al",
      "iveness" : "ive",
      "fulness" : "ful",
      "ousness" : "ous",
      "aliti" : "al",
      "iviti" : "ive",
      "biliti" : "ble",
      "logi" : "log"
    },

    step3list = {
      "icate" : "ic",
      "ative" : "",
      "alize" : "al",
      "iciti" : "ic",
      "ical" : "ic",
      "ful" : "",
      "ness" : ""
    },

    c = "[^aeiou]",          // consonant
    v = "[aeiouy]",          // vowel
    C = c + "[^aeiouy]*",    // consonant sequence
    V = v + "[aeiou]*",      // vowel sequence

    mgr0 = "^(" + C + ")?" + V + C,               // [C]VC... is m>0
    meq1 = "^(" + C + ")?" + V + C + "(" + V + ")?$",  // [C]VC[V] is m=1
    mgr1 = "^(" + C + ")?" + V + C + V + C,       // [C]VCVC... is m>1
    s_v = "^(" + C + ")?" + v;                   // vowel in stem

  function dummyDebug() {}

  function realDebug() {
    console.log(Array.prototype.slice.call(arguments).join(' '));
  }

  return function (w, debug) {
    var
      stem,
      suffix,
      firstch,
      re,
      re2,
      re3,
      re4,
      debugFunction,
      origword = w;

    if (debug) {
      debugFunction = realDebug;
    } else {
      debugFunction = dummyDebug;
    }

    if (w.length < 3) { return w; }

    firstch = w.substr(0,1);
    if (firstch == "y") {
      w = firstch.toUpperCase() + w.substr(1);
    }

    // Step 1a
    re = /^(.+?)(ss|i)es$/;
    re2 = /^(.+?)([^s])s$/;

    if (re.test(w)) {
      w = w.replace(re,"$1$2");
      debugFunction('1a',re, w);

    } else if (re2.test(w)) {
      w = w.replace(re2,"$1$2");
      debugFunction('1a',re2, w);
    }

    // Step 1b
    re = /^(.+?)eed$/;
    re2 = /^(.+?)(ed|ing)$/;
    if (re.test(w)) {
      var fp = re.exec(w);
      re = new RegExp(mgr0);
      if (re.test(fp[1])) {
        re = /.$/;
        w = w.replace(re,"");
        debugFunction('1b',re, w);
      }
    } else if (re2.test(w)) {
      var fp = re2.exec(w);
      stem = fp[1];
      re2 = new RegExp(s_v);
      if (re2.test(stem)) {
        w = stem;
        debugFunction('1b', re2, w);

        re2 = /(at|bl|iz)$/;
        re3 = new RegExp("([^aeiouylsz])\\1$");
        re4 = new RegExp("^" + C + v + "[^aeiouwxy]$");

        if (re2.test(w)) {
          w = w + "e";
          debugFunction('1b', re2, w);

        } else if (re3.test(w)) {
          re = /.$/;
          w = w.replace(re,"");
          debugFunction('1b', re3, w);

        } else if (re4.test(w)) {
          w = w + "e";
          debugFunction('1b', re4, w);
        }
      }
    }

    // Step 1c
    re = new RegExp("^(.*" + v + ".*)y$");
    if (re.test(w)) {
      var fp = re.exec(w);
      stem = fp[1];
      w = stem + "i";
      debugFunction('1c', re, w);
    }

    // Step 2
    re = /^(.+?)(ational|tional|enci|anci|izer|bli|alli|entli|eli|ousli|ization|ation|ator|alism|iveness|fulness|ousness|aliti|iviti|biliti|logi)$/;
    if (re.test(w)) {
      var fp = re.exec(w);
      stem = fp[1];
      suffix = fp[2];
      re = new RegExp(mgr0);
      if (re.test(stem)) {
        w = stem + step2list[suffix];
        debugFunction('2', re, w);
      }
    }

    // Step 3
    re = /^(.+?)(icate|ative|alize|iciti|ical|ful|ness)$/;
    if (re.test(w)) {
      var fp = re.exec(w);
      stem = fp[1];
      suffix = fp[2];
      re = new RegExp(mgr0);
      if (re.test(stem)) {
        w = stem + step3list[suffix];
        debugFunction('3', re, w);
      }
    }

    // Step 4
    re = /^(.+?)(al|ance|ence|er|ic|able|ible|ant|ement|ment|ent|ou|ism|ate|iti|ous|ive|ize)$/;
    re2 = /^(.+?)(s|t)(ion)$/;
    if (re.test(w)) {
      var fp = re.exec(w);
      stem = fp[1];
      re = new RegExp(mgr1);
      if (re.test(stem)) {
        w = stem;
        debugFunction('4', re, w);
      }
    } else if (re2.test(w)) {
      var fp = re2.exec(w);
      stem = fp[1] + fp[2];
      re2 = new RegExp(mgr1);
      if (re2.test(stem)) {
        w = stem;
        debugFunction('4', re2, w);
      }
    }

    // Step 5
    re = /^(.+?)e$/;
    if (re.test(w)) {
      var fp = re.exec(w);
      stem = fp[1];
      re = new RegExp(mgr1);
      re2 = new RegExp(meq1);
      re3 = new RegExp("^" + C + v + "[^aeiouwxy]$");
      if (re.test(stem) || (re2.test(stem) && !(re3.test(stem)))) {
        w = stem;
        debugFunction('5', re, re2, re3, w);
      }
    }

    re = /ll$/;
    re2 = new RegExp(mgr1);
    if (re.test(w) && re2.test(w)) {
      re = /.$/;
      w = w.replace(re,"");
      debugFunction('5', re, re2, w);
    }

    // and turn initial Y back to y
    if (firstch == "y") {
      w = firstch.toLowerCase() + w.substr(1);
    }


    return w;
  }
})();
//end porter

function tokenizeString(inputString) {
  // Using regular expression to split the string by non-word characters and digits
  // This will tokenize the string based on spaces, punctuation, etc., while excluding numbers
  return inputString.split(/\W+|\d+/).filter(Boolean);
}

firebaseConfig = {
  databaseURL: "https://search-engine-568ee-default-rtdb.europe-west1.firebasedatabase.app/",
};

function removeStop(query){
  let stopWords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',
'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once',
'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
 "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"];
 // Remove stop words from the query array
  let filteredQuery = [];
  for (let word of query) {
    if (!stopWords.includes(word.toLowerCase())) {
      filteredQuery.push(word);
    }
  }
  return filteredQuery;
}
  var currentWindow = 1;
  let terms,pagesList;

  async function homepageView(){
    currentWindow = 0;
    let data= await singleIndex();
    let data2= await singlepagesList();
    document.getElementById("content").innerHTML = homepage;
    // Load recent searches from localStorage
    let recentSearches = JSON.parse(localStorage.getItem("recentSearches")) || [];
    // Update recent searches section
    updateRecentSearches(recentSearches);

    document.querySelector('.popular-searches .popular-links').addEventListener('click', function(event) {
      // Check if the clicked element is an anchor tag
      if (event.target.tagName === 'A') {
        // Prevent the default link behavior
        event.preventDefault();
        // Call the search function with appropriate parameters
        search(1, event.target.innerText);
      }
    });
    document.getElementById("searchButton").addEventListener("click", search);
    document.getElementById("searchInput").addEventListener("keypress", function(event) {
      if (event.key === "Enter") {
        search();
      }
    });
  }

  function botView(){
    document.getElementById("bot").innerHTML = bot;
    const chatbotToggler = document.querySelector(".chatbot-toggler");
const closeBtn = document.querySelector(".close-btn");
const chatbox = document.querySelector(".chatbox");
const chatInput = document.querySelector(".chat-input textarea");
const sendChatBtn = document.querySelector(".chat-input span");

let userMessage = null; // Variable to store user's message
const API_KEY = "PASTE-YOUR-API-KEY"; // Paste your API key here
const inputInitHeight = chatInput.scrollHeight;

const createChatLi = (message, className) => {
    // Create a chat <li> element with passed message and className
    const chatLi = document.createElement("li");
    chatLi.classList.add("chat", `${className}`);
    let chatContent = className === "outgoing" ? `<p></p>` : `<span class="material-symbols-outlined">smart_toy</span><p></p>`;
    chatLi.innerHTML = chatContent;
    chatLi.querySelector("p").textContent = message;
    return chatLi; // return chat <li> element
}

function removeLetter(str, letterToRemove) {
  // Remove from beginning
  if (str.charAt(0) === letterToRemove) {
    str = str.substring(1);
  }

  // Remove from end
  if (str.charAt(str.length - 1) === letterToRemove) {
    str = str.substring(0, str.length - 1);
  }

  return str;
}



function botSearch(searchTerm) {
    let tokenizedQuery = tokenizeString(searchTerm.toLowerCase());
    tokenizedQuery=removeStop(tokenizedQuery);
    console.log(tokenizedQuery);
    let stemmedQuery = tokenizedQuery.map((word) => stemmer(word));
    const links = fetchAndIntersectPages(stemmedQuery);
    return links;
}

const generateResponse =  async (chatElement) => {

    const messageElement = chatElement.querySelector("p");
      let searchResult = botSearch(userMessage);

    let res="";
    await google.colab.kernel
      .invokeFunction("notebook.askChat", [userMessage], {})
      .then(function (result) {
        rawText = result.data["text/plain"].trim();

        if (rawText === "None") {

           if(searchResult.length > 0) {

                console.log("inside length");
                const sortedResults = searchResult.sort((a, b) => sortSearchResults(a, b, categories, userMessage));
               res=sortedResults[0];
                rawText = res.value.description + "$" + res.value.link;

            }

          else {
            rawText= "i'm sorry, i don't have information about it, can I help with anything else?";
          }

        }


        var rawTextFormatted = removeLetter(rawText,"'");
        rawTextSplit = rawTextFormatted.split("$");

        messageElement.textContent = rawTextSplit[0];

        if (rawTextSplit.length > 1) {

          var lineBreak = document.createElement("br");
          aliLink = document.createElement("a");
          aliLink.href = rawTextSplit[1];
          aliLink.target = "_blank";
          aliLink.textContent = "For More Information..";
          messageElement.appendChild(lineBreak);
          messageElement.appendChild(aliLink);

        }

      })
      .catch(function (error) {
           console.log(error)
        messageElement.classList.add("error");
        messageElement.textContent =
          "Oops! Something went wrong. Please try again.";
      })
      .finally(() => chatbox.scrollTo(0, chatbox.scrollHeight));

}

const handleChat = async () => {
    userMessage = chatInput.value.trim(); // Get user entered message and remove extra whitespace
    if(!userMessage) return;

    // Clear the input textarea and set its height to default
    chatInput.value = "";
    chatInput.style.height = `${inputInitHeight}px`;

    // Append the user's message to the chatbox
    chatbox.appendChild(createChatLi(userMessage, "outgoing"));
    chatbox.scrollTo(0, chatbox.scrollHeight);

    //setTimeout(() => {
        // Display "Thinking..." message while waiting for the response
        const incomingChatLi = createChatLi("Thinking...", "incoming");
        chatbox.appendChild(incomingChatLi);
        chatbox.scrollTo(0, chatbox.scrollHeight);
        await generateResponse(incomingChatLi);
   // }, 600);
}

chatInput.addEventListener("input", () => {
    // Adjust the height of the input textarea based on its content
    chatInput.style.height = `${inputInitHeight}px`;
    chatInput.style.height = `${chatInput.scrollHeight}px`;
});

chatInput.addEventListener("keydown", async (e) => {
    // If Enter key is pressed without Shift key and the window
    // width is greater than 800px, handle the chat
    if(e.key === "Enter" && !e.shiftKey && window.innerWidth > 800) {
        e.preventDefault();
        await handleChat();
    }
});

sendChatBtn.addEventListener("click", handleChat);
closeBtn.addEventListener("click", () => document.body.classList.remove("show-chatbot"));
chatbotToggler.addEventListener("click", () => document.body.classList.toggle("show-chatbot"));

}

firebase.initializeApp(firebaseConfig);
homepageView();
botView();

  async function search(type = 0, text = "") {
    // Get the search query
    var searchQuery = document.getElementById("searchInput").value;
    if (type === 1) {
        searchQuery = text;
    }

    // Clear the content of the content div
    document.getElementById("content").innerHTML = '<div class="loaderContainer"><div class="loader"><div class="spinner"></div></div></div>';
    let tokenizedQuery = tokenizeString(searchQuery.toLowerCase());
    if (tokenizedQuery.length != 0) {
        // Save the search query in recent searches
        saveRecentSearch(searchQuery);
        // Update search count for the whole query
        updateSearchCountInFirebase(tokenizedQuery.join("_"));
    }
    tokenizedQuery=removeStop(tokenizedQuery);
    let stemmedQuery = tokenizedQuery.map((word) => stemmer(word));
    const links = fetchAndIntersectPages(stemmedQuery);

    // Create HTML for displaying search results
    var resultsHtml = `<div class="searchResultContainer"><p>The search results are sorted based on their category(which is diplayed near the results' titles) and the similiraty between the query and the page's title and description
    </p><br><h2>Search Results</h2><div class="scrollable"><div id="searchResults"></div></div></div>`;

    // Display search results in the content div
    document.getElementById("content").innerHTML = searchDiv + resultsHtml;
    createSearchResultElements(links,searchQuery);

    // Add event listeners for search button and Enter key press
    document.getElementById("searchButton").addEventListener("click", search);
    document.getElementById("searchInput").addEventListener("keypress", function(event) {
        if (event.key === "Enter") {
            search();
        }
    });

    // Set the search input value
    document.getElementById("searchInput").value = searchQuery;
}

const categories = [ { name: "knowledge", keywords: ["knowledge"] },
    { name: "Docs", keywords: ["docs", "documentation", "help"] },
    { name: "Community", keywords: ["blog", "article", "post"] },
    { name: "Products", keywords: ["product"] },
    { name: "Solution", keywords: ["solutions"] },
];

function getCategoryForLink(link) {
    for (const category of categories) {
        for (const keyword of category.keywords) {
            if (link.toLowerCase().includes(keyword)) {
                return category.name;
            }
        }
    }
    return "Other";
}

// Helper function to count the number of words in a string that match the query
function countWordsMatched(str, words) {
  let count = 0;
  for (const word of words) {
    if (str.includes(word)) {
      count++;
    }
  }
  return count;
}

function sortSearchResults(a, b, categories, query) {
    const categoryA = getCategoryForLink(a.value.link);
    const categoryB = getCategoryForLink(b.value.link);
    const isDefinedCategoryA = categories.some((c) => c.name === categoryA);
    const isDefinedCategoryB = categories.some((c) => c.name === categoryB);

    // If both results belong to defined categories, sort based on category order
    if (isDefinedCategoryA && isDefinedCategoryB) {
        const categoryOrderA = categories.findIndex((c) => c.name === categoryA);
        const categoryOrderB = categories.findIndex((c) => c.name === categoryB);
        if (categoryOrderA !== categoryOrderB) {
            return categoryOrderA - categoryOrderB;
        }

        // Calculate word matches for title and description
        const queryWords = query.toLowerCase().split(' ');
        const titleMatchesA = countWordsMatched(a.value.title.toLowerCase(), queryWords);
        const titleMatchesB = countWordsMatched(b.value.title.toLowerCase(), queryWords);
        const descriptionMatchesA = countWordsMatched(a.value.description.toLowerCase(), queryWords);
        const descriptionMatchesB = countWordsMatched(b.value.description.toLowerCase(), queryWords);

        // Sort based on the number of word matches in title and description
        const totalMatchesA = titleMatchesA + descriptionMatchesA;
        const totalMatchesB = titleMatchesB + descriptionMatchesB;
        return totalMatchesB - totalMatchesA; // Sort in descending order of matches
    }

    // If only one result belongs to a defined category, prioritize that result
    if (isDefinedCategoryA && !isDefinedCategoryB) {
        return -1;
    }
    if (!isDefinedCategoryA && isDefinedCategoryB) {
        return 1;
    }

    // If neither result belongs to a defined category, sort alphabetically
    return a.value.link.localeCompare(b.value.link);
}


// Function to dynamically create search result elements
function createSearchResultElements(results, query) {
    const searchResultsContainer = document.getElementById("searchResults");
    searchResultsContainer.innerHTML = "";

    // Sort the results based on whether they belong to a defined category or not
    const sortedResults = results.sort((a, b) => sortSearchResults(a, b, categories, query));
    sortedResults.forEach((result) => {
        const resultElement = document.createElement("div");
        resultElement.className = "searchResultDiv";

        const titleElement = document.createElement("a");
        titleElement.classList.add("title");
        titleElement.href = result.value.link;
        titleElement.textContent = result.value.title;
        titleElement.target = "_blank";
        resultElement.appendChild(titleElement);

        const categoryElement = document.createElement("span");
        categoryElement.classList.add("category");
        const category = getCategoryForLink(result.value.link);
        categoryElement.textContent = category;
        resultElement.appendChild(categoryElement);

        const descriptionElement = document.createElement("p");
        descriptionElement.textContent = result.value.description;
        descriptionElement.classList.add("description");
        resultElement.appendChild(descriptionElement);

        searchResultsContainer.appendChild(resultElement);
    });
}




// Function to update search count for a phrase in the Firebase database
function updateSearchCountInFirebase(phrase) {
    const database = firebase.database();
    const indexRef = database.ref("/search_index");

    // Query the index node to find the entry with the specified phrase
    indexRef.child(phrase).once("value", (snapshot) => {
        const entry = snapshot.val();
        if (entry) {
            // If the phrase exists, increment the count
            const newCount = entry.count ? entry.count + 1 : 1;
            indexRef.child(phrase).update({ count: newCount });
        } else {
            // If the phrase does not exist, create a new entry
            indexRef.child(phrase).set({ count: 1 });
        }
    });
}

  function saveRecentSearch(searchQuery) {
    let recentSearches = getRecentSearches();

    // Add the new search query to the recent searches
    recentSearches.unshift(searchQuery);

    // Ensure only 5 recent searches are stored
    if (recentSearches.length > 5) {
      recentSearches.pop();
    }

    // Save recent searches to localStorage
    localStorage.setItem("recentSearches", JSON.stringify(recentSearches));
  }

  function getRecentSearches() {
    return JSON.parse(localStorage.getItem("recentSearches")) || [];
  }

  function updateRecentSearches(recentSearches) {
    let recentSearchesContainer = document.querySelector(".rect-searches .popular-links");
    recentSearchesContainer.innerHTML = "";
    let id=1;
    recentSearches.forEach(function(searchQuery) {
      let link = document.createElement("a");
      link.id=id;
      id++;
      link.href = "#";
      link.textContent = searchQuery;
      link.addEventListener("click", function(event){event.preventDefault();search(1,searchQuery)});
      recentSearchesContainer.appendChild(link);
    });
  }

  function backToHomepage(){
    homepageView();
  }





var buttonsPerPage = 12;
var currentPage = 1;
var totalButtons = 50; // Total number of buttons


let currentTerm;

///////////////////////////////////LOGIN/////////////////////////////////

const adminUsername = "admin";
const adminPassword = "password123";
const loggedInKey = "false";

// Function to check if the user is logged in
function isLoggedIn() {
  return localStorage.getItem(loggedInKey) === "true";
}

// Function to set the login status in localStorage
function setLoggedIn(status) {
  localStorage.setItem(loggedInKey, status.toString());
}

// Function to clear the login status from localStorage and return to home page
function logout() {
    localStorage.removeItem(loggedInKey);
    document.getElementById("loginModal").style.display = "none";
    document.getElementById("logout").style.display = "none"; // Hide logout button
    document.getElementById("loginButton").style.display = "block";
    homepageView(); // Return to home page
}

// Open the login modal when the admin button is clicked
document.getElementById("admin").addEventListener("click", () => {
    if (isLoggedIn() && currentWindow === 0) {
          document.getElementById("loginModal").style.display = "none";
          adminViewContent();
    }
    else{
       document.getElementById("loginModal").style.display = "block";
    }

});


// Close the login modal when the close button is clicked
document.getElementsByClassName("close")[0].addEventListener("click", () => {
  document.getElementById("loginModal").style.display = "none";
});

// Handle login form submission
document.getElementById("loginForm").addEventListener("submit", async (event) => {
  event.preventDefault();

  const username = document.getElementById("username").value;
  const password = document.getElementById("password").value;

  // Check if the entered credentials match the admin credentials
  if (username === adminUsername && password === adminPassword) {
    // Set loggedIn flag in localStorage
    setLoggedIn(true);
    // Call adminViewContent function
    adminViewContent();
    document.getElementById("loginModal").style.display = "none";
    document.getElementById("loginButton").style.display = "none";
    document.getElementById("logout").style.display = "block"; // Show logout button
  } else {
    alert("Invalid username or password. Please try again.");
  }
});

// Handle logout button click
document.getElementById("logout").addEventListener("click", () => {
    logout();
});

async function adminViewContent(){
  currentWindow=1;
  currentPage = 1;
  var adminContainer = document.getElementById('content');
  adminContainer.innerHTML = `
  <div class="container">
    <div class="div1">
          <div  id="terms"></div>

                  <div id="navigationButtons">
                  <button id="prev" onclick="prevPage()">prev</button>
                  <button id="next" onclick="nextPage()">next</button>
              </div>
    </div>

            <div class="div2" id="links"></div>
  </div>
   `;
  totalButtons=terms.length;
  console.log(terms);
  createButtons();
}


///////////////////////////////////////////////////////

function fetchIndexData() {
  return new Promise((resolve, reject) => {
    const database = firebase.database().ref("/index");

    database.once('value')
      .then(snapshot => {
        const data = snapshot.val();
        resolve(data); // Resolve the promise with data
      })
      .catch(error => {
        console.error('Error fetching data:', error);
        reject(error); // Reject the promise with error
      });
  });
}

//added single index
async function singleIndex(){
  if(!terms){
    document.getElementById("content").innerHTML='<div class="loaderContainer"><div class="loader"><div class="spinner"></div></div></div>';
    let data = await fetchIndexData();
    terms = Object.entries(data).map(([key, value]) => ({ key, value }));
  }
  return terms;
}

function fetchPagesData() {
  return new Promise((resolve, reject) => {
    const database = firebase.database().ref("/pages");

    database.once('value')
      .then(snapshot => {
        const data = snapshot.val();
        resolve(data); // Resolve the promise with data
      })
      .catch(error => {
        console.error('Error fetching data:', error);
        reject(error); // Reject the promise with error
      });
  });
}

async function singlepagesList(){
  if(!pagesList){
    document.getElementById("content").innerHTML='<div class="loaderContainer"><div class="loader"><div class="spinner"></div></div></div>';
    let data = await fetchPagesData();
    pagesList = data;
  }
  return pagesList;
}

function fetchAndIntersectPages(stemmedQuery) {//improved search
  let intersection = new Set();

  if (stemmedQuery.length === 0) {
    return [];
  }

  // Fetch pages for the first word to initialize the intersection set
  const firstWordPages = fetchTermPages(stemmedQuery[0]);
  intersection = new Set(Object.keys(firstWordPages));

  // Iterate over the remaining words in the stemmed query
  for (let i = 1; i < stemmedQuery.length; i++) {
    const word = stemmedQuery[i];
    const pages = fetchTermPages(word);

    // Get the keys of the current word's pages
    const currentPageKeys = new Set(Object.keys(pages));

    // Perform intersection with the existing intersection set
    intersection = new Set([...intersection].filter(pageKey => currentPageKeys.has(pageKey)));
  }

  // Convert the intersection Set to an array of objects with key and value
  const intersectionArray = Array.from(intersection).map(key => ({
    key,
    value: firstWordPages[key] // Get the value for each key from the first word's pages
  }));

  return intersectionArray;
}


function fetchTermPages(term) {
  let termLinks=[];
  for (var i = 0; i < terms.length; i++) {
    if (terms[i].value.term == term) {
      termLinks = terms[i].value.DocIDs;
      break; // Exit loop once termLinks are found
    }
  }
        const data = pagesList;
        // Filter data based on termLinks
        const filteredPages = {};
        termLinks.forEach(docID => {
          if (data.hasOwnProperty(docID)) {
            filteredPages[docID] = data[docID];
          }
        });
        console.log(data);
        return filteredPages;
}


function createButtons() {
    var buttonContainer = document.getElementById('terms');
    buttonContainer.innerHTML = '';
    var startIndex = (currentPage - 1) * buttonsPerPage;
    var endIndex = Math.min(startIndex + buttonsPerPage, totalButtons);

    var container = document.createElement('div');
    container.className = 'buttonContainer';

    for (var i = startIndex; i < endIndex; i++) {
        var button = document.createElement('button');
        button.className = 'indexButton';
        let term=terms[i].value.term;
        button.innerText =  (term);
        button.addEventListener("click", function(event){displayLinks(term)});
        container.appendChild(button);
    }

    buttonContainer.appendChild(container);

}

function createLinksList(listItems, targetDivId) {
  // Create a new ul element
  var title = document.createElement('h2');
  title.textContent = currentTerm + ":";
  title.id = "termTitle";
  var ul = document.createElement('ul');

  var deleteTermbutton = document.createElement('button');
  deleteTermbutton.textContent = 'delete all';
  deleteTermbutton.id = 'delete all';
  deleteTermbutton.addEventListener('click', function(event) { deleteTerm(); });

  let id = 0;
  listItems.forEach(function(item) {
      var li = document.createElement('li');
      li.textContent = item.value.link;
      let linkID = "link " + id;
      li.id = linkID;
      var button = document.createElement('button');
      button.textContent = 'delete';
      button.addEventListener('click', function(event) { deleteLink(item.key, item.value.link, linkID); });
      li.appendChild(button);
      ul.appendChild(li);
      id++;
  });

  var targetDiv = document.getElementById(targetDivId);
  targetDiv.innerHTML = '';

  var scrollableDiv = document.createElement('div');
  scrollableDiv.className = 'scrollable'; // Add a class for styling

  scrollableDiv.appendChild(deleteTermbutton);
  scrollableDiv.appendChild(title);
  scrollableDiv.appendChild(ul);

  targetDiv.appendChild(scrollableDiv);
}


function deleteLink(id,link,linkID){
  const database = firebase.database();

  // Reference to the 'index' node in the database
  const indexRef = database.ref('/index');

  // Query the index node to find the entry with the specified term
  indexRef.orderByChild('term').equalTo(currentTerm).once('value', snapshot => {
    snapshot.forEach(childSnapshot => {
      const key = childSnapshot.key;
      const entry = childSnapshot.val();

      // Check if the entry has the specified docId in its DocIDs array
      if (entry.DocIDs && entry.DocIDs.includes(id)) {
        // Remove the id from the DocIDs array
        const updatedDocIDs = entry.DocIDs.filter(docId => docId !== id);

        // Update the entry in the database
        indexRef.child(key).update({ DocIDs: updatedDocIDs })
          .then(() => {
            console.log(`Removed docId ${id} from term ${currentTerm}`);
            let linkElement= document.getElementById(linkID);
            linkElement.remove();
            let termLinks;
            for (var i = 0; i < terms.length; i++) {
              if (terms[i].value.term == currentTerm) {
                terms[i].value.DocIDs=updatedDocIDs;
                if(updatedDocIDs.length===0){
                  terms.splice(i, 1); // Remove the term at index i
                  totalButtons=terms.length;
                }
                break; // Exit loop once termLinks are found
              }
            }
            if (updatedDocIDs.length === 0) {
              removeEntryFromIndex(key);//remove term from db
              var totalPages = Math.ceil(totalButtons / buttonsPerPage);
              if (currentPage > totalPages) {
                  currentPage--;
              }
              document.getElementById("termTitle").remove();//remove title of the term
              document.getElementById("delete all").remove();//remove button of the term
              createButtons();//update display
            }
          })
          .catch(error => {
            console.error('Error removing docId:', error);
          });
      } else {
        console.log(`DocId ${id} not found in term ${currentTerm}`);
      }
    });
  })
  .catch(error => {
    console.error('Error fetching data:', error);
  });
}

function deleteTerm(){
  for (var i = 0; i < terms.length; i++) {
    if (terms[i].value.term == currentTerm) {
        removeEntryFromIndex(terms[i].key);//remove term from db
        terms.splice(i, 1); // Remove the term at index i
        totalButtons=terms.length;
      break; // Exit loop once termLinks are found
    }
  }
    var totalPages = Math.ceil(totalButtons / buttonsPerPage);
    if (currentPage > totalPages) {
        currentPage--;
    }
    var targetDiv = document.getElementById("links");
    // Clear the target div before loading the list
    targetDiv.innerHTML = '';
    createButtons();//update display
}

function removeEntryFromIndex(key) {
  const database = firebase.database();
  const indexRef = database.ref('/index');

  // Remove the entry with the specified key
  indexRef.child(key).remove()
    .then(() => {
      console.log(`Entry with key ${key} removed from index.`);
    })
    .catch(error => {
      console.error('Error removing entry from index:', error);
    });
}

async function displayLinks(term){
  currentTerm=term;
  let data= fetchTermPages(term);
  let links=Object.entries(data).map(([key, value]) => ({ key, value }));
  createLinksList(links,"links");
}

function prevPage() {
    if (currentPage > 1) {
        currentPage--;
        createButtons();
    }
    else{
      alert("you're in the first page");
    }
}

function nextPage() {
    var totalPages = Math.ceil(totalButtons / buttonsPerPage);
    if (currentPage < totalPages) {
        currentPage++;
        createButtons();
    }
	  else{
      alert("you're in the last page");
    }
}


async function fetchSearchIndexData() {
  const database = firebase.database();
  const indexRef = database.ref("/search_index");

  return new Promise((resolve, reject) => {
      indexRef.once("value", (snapshot) => {
          const data = snapshot.val();
          if (data) {
              resolve(data);
          } else {
              reject("No data found in search_index");
          }
      });
  });
}

function replaceUnderscoresWithSpaces(str) {
  return str.replace(/_/g, ' ');
}

function displaySearchIndexTable() {
    fetchSearchIndexData()
        .then((data) => {
            const table = document
                .getElementById("searchIndexTable")
                .getElementsByTagName("tbody")[0];
            let totalCount = 0;

            const sortedData = Object.entries(data).sort(
                (a, b) => b[1].count - a[1].count
            );

            sortedData.forEach(([phrase, entry]) => {
                totalCount += entry.count;
            });

            sortedData.forEach(([phrase, entry]) => {
                const row = table.insertRow();
                const cell1 = row.insertCell(0);
                const cell2 = row.insertCell(1);
                const cell3 = row.insertCell(2);

                cell1.textContent = replaceUnderscoresWithSpaces(phrase);
                cell2.textContent = entry.count;

                const percentage = ((entry.count / totalCount) * 100).toFixed(
                    2
                );
                cell3.textContent = percentage + "%";
            });

            createChart(sortedData, totalCount);
        })
        .catch((error) => {
            console.error("Error fetching data:", error);
            alert("Error fetching data. Please try again later.");
        });
}

function createChart(sortedData, totalCount) {
    const phrases = [];
    const counts = [];
    const percentages = [];

    const topPhrases = sortedData.slice(0, 10);

    topPhrases.forEach(([phrase, entry]) => {
        phrases.push(replaceUnderscoresWithSpaces(phrase));
        counts.push(entry.count);

        const percentage = ((entry.count / totalCount) * 100).toFixed(2);
        percentages.push(percentage);
    });

    const ctx = document.getElementById("myChart").getContext("2d");
    const myChart = new Chart(ctx, {
        type: "bar",
        data: {
            labels: phrases,
            datasets: [
                {
                    label: "Number of searches",
                    data: counts,
                    backgroundColor: "rgba(54, 162, 235, 0.6)",
                    borderColor: "rgba(54, 162, 235, 1)",
                    borderWidth: 1,
                },
            ],
        },
        options: {
            scales: {
                yAxes: [
                    {
                        ticks: {
                            beginAtZero: true,
                        },
                    },
                ],
            },
        },
    });

    const pieCtx = document.getElementById("myPieChart").getContext("2d");
    const myPieChart = new Chart(pieCtx, {
        type: "pie",
        data: {
            labels: phrases,
            datasets: [
                {
                    label: "Percentage",
                    data: percentages,
                    backgroundColor: [
                        "rgba(255, 99, 132, 0.6)",
                        "rgba(54, 162, 235, 0.6)",
                        "rgba(255, 206, 86, 0.6)",
                        "rgba(75, 192, 192, 0.6)",
                        "rgba(153, 102, 255, 0.6)",
                        "rgba(255, 159, 64, 0.6)",
                        "rgba(255, 99, 132, 0.6)",
                        "rgba(54, 162, 235, 0.6)",
                        "rgba(255, 206, 86, 0.6)",
                        "rgba(75, 192, 192, 0.6)",
                    ],
                    borderColor: [
                        "rgba(255, 99, 132, 1)",
                        "rgba(54, 162, 235, 1)",
                        "rgba(255, 206, 86, 1)",
                        "rgba(75, 192, 192, 1)",
                        "rgba(153, 102, 255, 1)",
                        "rgba(255, 159, 64, 1)",
                        "rgba(255, 99, 132, 1)",
                        "rgba(54, 162, 235, 1)",
                        "rgba(255, 206, 86, 1)",
                        "rgba(75, 192, 192, 1)",
                    ],
                    borderWidth: 1,
                },
            ],
        },
        options: {
            responsive: true,
            legend: {
                position: "right",
            },
        },
    });
}




function linksCounter() {
    const termCounts = {};

    // Iterate through each term in the global terms array
    terms.forEach((term) => {
        const termValue = term.value;
        const termName = termValue.term;
        const termLinks = termValue.DocIDs || [];

        // Count the number of links for this term
        const linkCount = termLinks.length;

        // Update the termCounts object with the link count
        termCounts[termName] = linkCount;
    });

    // Return the termCounts object
    return termCounts;
}


function updateTop10Terms(termCounts) {
    // Convert termCounts object to an array for sorting
    const termsArray = Object.keys(termCounts).map((termKey) => {
        return {
            term: termKey,
            count: termCounts[termKey], // Count from the termCounts object
        };
    });

    // Sort terms by count in descending order
    termsArray.sort((a, b) => b.count - a.count);

    // Take the top 10 terms
    const top10Terms = termsArray.slice(0, 10);

    // Add the 'name' attribute to each term in top10Terms
    const top10TermsWithNames = top10Terms.map(({ term, count }) => {
        return { term, count, name: term }; // Adding 'name' attribute
    });

    // Log the top 10 terms with their counts
    console.log("Top 10 terms:");
    top10TermsWithNames.forEach(({ term, count, name }, index) => {
        console.log(`${index + 1}. ${name}: ${count} links`);
    });

    // Call createGraph with the top 10 terms and their counts
    createGraph(top10TermsWithNames);
}


function createGraph(top10Terms) {
    const chartLabels = top10Terms.map(({ name }) => name); // Using 'name' for chart labels
    const chartData = top10Terms.map(({ count }) => count);

    const ctx = document.getElementById("myBarChart").getContext("2d");

    const chart = new Chart(ctx, {
        type: "bar",
        data: {
            labels: chartLabels,
            datasets: [
                {
                    label: "Count of links for the term",
                    data: chartData,
                    backgroundColor: "rgba(54, 162, 235, 0.6)",
                    borderColor: "rgba(54, 162, 235, 1)",
                    borderWidth: 1,
                },
            ],
        },
        options: {
            scales: {
                yAxes: [
                    {
                        ticks: {
                            beginAtZero: true,
                            precision: 0,
                        },
                        scaleLabel: {
                            display: true,
                            labelString: "Count",
                        },
                    },
                ],
                xAxes: [
                    {
                        scaleLabel: {
                            display: true,
                            labelString: "Terms",
                        },
                    },
                ],
            },
            plugins: {
                legend: {
                    display: false,
                },
            },
        },
    });
}

function goToStatsPage() {
  document.getElementById("content").innerHTML = statsScreen;
  displaySearchIndexTable();
  let termcounters=linksCounter();
  updateTop10Terms(termcounters);
}





        """)

    def render(self):
        output = self.css.render()
        output += self.html.render()
        output += self.js.render()
        return output


# Creating an instance of the Main class
main = Main()

# Rendering the output
output = main.render()

# Displaying the HTML page
from IPython.display import display, HTML
display(HTML(output))